# Apply Mass Balance Functions
This script is by Frowin; I added Memote at the end

## Imports

In [26]:
import cobra
from cobra.io import write_sbml_model, read_sbml_model
from cobra import Reaction, Metabolite
import os
import pandas as pd
import ast
from collections import defaultdict
import subprocess
import json

# import functions from other notebooks (have to be in the same directory!)
import import_ipynb

## Paths

In [4]:
# Path to your working directory
project_dir = "/home/lisa/Dokumente/Programmierung/"

# Path to the model files
model_path = project_dir+ "Models/before_mass_balance/"

# Path to the directory where you want to save the results
save_path = project_dir + "Models/mass_balance/"

Import the results of the Identify_imbalanced_reactions script

In [5]:
imbalanced_reactions = pd.read_csv(save_path+'Imbalanced_reactions.csv', sep='\t')

Inspect the table of imbalanced reactions

In [6]:
imbalanced_reactions

,Unnamed: 0,Reaction,Occurences,Model IDs
0,0,CMCBTFL,7,"['AA5.xml', 'AA4.xml', 'AA6.xml', 'AA1.xml', '..."
1,1,SALCHS4FEabcpp,7,"['AA5.xml', 'AA4.xml', 'AA6.xml', 'AA1.xml', '..."
2,2,CMCBTFabcpp,3,"['AA4.xml', 'AA7.xml', 'AA2.xml']"
3,3,DHBSZ3FEabcpp,4,"['AA4.xml', 'AA6.xml', 'AA3.xml', 'AA2.xml']"


Create dictionary containing each imbalanced reaction as a key and the models this reaction is imbalanced in as a list

In [7]:
reaction_dict = {}

# Iterate through all rows of the imbalanced_reactions DataFrame
for row in imbalanced_reactions.iterrows():

    # The List of models is saved as a string in the DataFrame
    # Convert the string representation of the list into an actual list using ast.literal_eval
    reaction_dict[row[1][1]] = ast.literal_eval(row[1][3])

# Combine CMCBTFabcpp and CMCBTFL into one entry since the function for these is the same
reaction_dict['CMCBTF'] = reaction_dict['CMCBTFabcpp'] + list(set(reaction_dict['CMCBTFL']) - set(reaction_dict['CMCBTFabcpp']))
    

/tmp/ipykernel_80447/1090651779.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  reaction_dict[row[1][1]] = ast.literal_eval(row[1][3])


## Functions to fix imbalanced networks

In [8]:
def fix_cmcbtf(model):
        
    # Change the formula and charge of fcmcbtt_c in all models
    model.metabolites.get_by_id('fcmcbtt_c').formula = 'C33FeH48N5O13'
    model.metabolites.get_by_id('fcmcbtt_c').charge = 3
    
    # Change the charge of fcmcbtt_p and _e if they are present in the model
    if 'fcmcbtt_p' in model.metabolites:
        model.metabolites.get_by_id('fcmcbtt_p').charge = 3
    if 'fcmcbtt_e' in model.metabolites:
        model.metabolites.get_by_id('fcmcbtt_e').charge = 3

    # Subtract protons from the reactions CMCBTFR1 and CMCBTFU if they are present in the model
    if 'CMCBTFR1' in model.reactions:
        model.reactions.get_by_id('CMCBTFR1').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if 'CMCBTFR2' in model.reactions:
        model.reactions.get_by_id('CMCBTFR2').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if 'CMCBTFU' in model.reactions:
        model.reactions.get_by_id('CMCBTFU').subtract_metabolites({model.metabolites.get_by_id('h_c'): 1})

    return model

In [9]:
def fix_dhbsz3feabcpp(model):

    # Change the formula and charge of fe3dhbzs3_c
    model.metabolites.get_by_id('fe3dhbzs3_c').charge = 3
    model.metabolites.get_by_id('fe3dhbzs3_c').formula = 'C30FeH28N3O16'

    # Change the charge of fe3dhbzs3_p and _e
    if 'fe3dhbzs3_p' in model.metabolites:
        model.metabolites.get_by_id('fe3dhbzs3_p').charge = 3
        model.metabolites.get_by_id('fe3dhbzs3_e').charge = 3

    if 'feenter_c' in model.metabolites:
        model.metabolites.get_by_id('feenter_c').charge = 3

    # Subtract protons from the right side of reaction FE3DHBZS3R, FEDHBZS3R1 and FEDHBZS3R2
    if "FE3DHBZS3R" in model.reactions:
        model.reactions.get_by_id('FE3DHBZS3R').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "FEDHBZS3R1" in model.reactions:
        model.reactions.get_by_id('FEDHBZS3R1').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "FEDHBZS3R2" in model.reactions:
        model.reactions.get_by_id('FEDHBZS3R2').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "FEDHBZS3R3" in model.reactions:
        model.reactions.get_by_id('FEDHBZS3R3').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "FEENTERES" in model.reactions:
        model.reactions.get_by_id('FEENTERES').subtract_metabolites({model.metabolites.get_by_id('h_c'): -1})

    # Return the model
    return model

In [10]:
def fix_man6gpts(model):

    # Deleting objects 
    # This wil delete all listed reactions and due to the 'True' argument also all resulting orphan metabolites. In this case only 'man6pglyc_e'
    model.remove_reactions(['MAN6Gpts', 'EX_man6pglyc_e'], True)

    # Create objects to be added to the model

    # Metabolites:

    # Metabolite manglyc_p
    manglyc_p = Metabolite(
        'manglyc_p',
        formula='C9H15O9',
        name='2(alpha-D-Mannosyl)-D-glycerate',
        compartment='p',
        charge =-1)
    
    # Metabolite manglyc_e
    manglyc_e = Metabolite(
        'manglyc_e',
        formula='C9H15O9',
        name='2(alpha-D-Mannosyl)-D-glycerate',
        compartment='e',
        charge =-1)
    
    # Reactions:

    # Reaction MANGLYCptspp
    MANGLYCptspp = Reaction('MANGLYCptspp')
    MANGLYCptspp.name = '2-O-alpha-mannosyl-D-glycerate transport via PEP:Pyr PTS (periplasm)'
    MANGLYCptspp.subsystem = ''
    MANGLYCptspp.lower_bound = 0.0 
    MANGLYCptspp.upper_bound = 1000.0  
    MANGLYCptspp.add_metabolites({
        manglyc_p: -1,
        model.metabolites.get_by_id('pep_c'): -1,
        model.metabolites.get_by_id('man6pglyc_c'): 1,
        model.metabolites.get_by_id('pyr_c'): 1
    })

    # Reaction MANGLYCtex
    MANGLYCtex = Reaction('MANGLYCtex')
    MANGLYCtex.name = '2-O-alpha-mannosyl-D-glycerate transport via diffusion (extracellular to periplasm)'
    MANGLYCtex.subsystem = ''
    MANGLYCtex.lower_bound = -1000.0 
    MANGLYCtex.upper_bound = 1000.0
    MANGLYCtex.add_metabolites({
        manglyc_e: -1,
        manglyc_p: 1
    })  

    # Reaction EX_manglyc_e
    EX_manglyc_e = Reaction('EX_manglyc_e')
    EX_manglyc_e.name = '2(alpha-D-Mannosyl)-D-glycerate exchange'
    EX_manglyc_e.subsystem = ''
    EX_manglyc_e.lower_bound = 0.0 
    EX_manglyc_e.upper_bound = 1000.0  
    EX_manglyc_e.add_metabolites({
        manglyc_e: -1
    })

    # Add reactions to the model
    model.add_reactions([MANGLYCptspp, MANGLYCtex, EX_manglyc_e])

    # Apply changes
    model.metabolites.get_by_id('man6pglyc_c').charge = -3
    model.metabolites.get_by_id('man6p_c').charge = -2

    # Return the modified model
    return model

In [11]:
def fix_mcbtfabcpp(model):
    
    # Change the formula and charge of fe3mcbtt_c
    model.metabolites.get_by_id('fe3mcbtt_c').charge = 3
    model.metabolites.get_by_id('fe3mcbtt_c').formula = 'C47FeH77N5O10'

    # Change the charge of fe3mcbtt_p and _e
    model.metabolites.get_by_id('fe3mcbtt_p').charge = 3
    model.metabolites.get_by_id('fe3mcbtt_e').charge = 3

    # Subtract protons from the right side of reaction MCBTFR1
    model.reactions.get_by_id('MCBTFR1').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})

    return model

In [12]:
def fix_salchs4feabcpp(model):

    # Change the formula and charge of salchs4fe_c
    model.metabolites.get_by_id('salchs4fe_c').charge = 3
    model.metabolites.get_by_id('salchs4fe_c').formula = 'C42FeH46N3O25'

    # Change the charge of salchs4fe_p and _e
    if 'salchs4fe_p' in model.metabolites:
        model.metabolites.get_by_id('salchs4fe_p').charge = 3
        model.metabolites.get_by_id('salchs4fe_e').charge = 3

    # Subtract protons from the right side of reaction SALCHS4FER1, SALCHS4FER2 and SALCHS4FER3
    if "SALCHS4FER1" in model.reactions:
        model.reactions.get_by_id('SALCHS4FER1').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "SALCHS4FER2" in model.reactions:
        model.reactions.get_by_id('SALCHS4FER2').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})
    if "SALCHS4FER3" in model.reactions:
        model.reactions.get_by_id('SALCHS4FER3').subtract_metabolites({model.metabolites.get_by_id('h_c'): 2})

    return model

In [13]:
def fix_uaccpts(model):
    
    # Change the formula and charge of uaccg_e
    model.metabolites.get_by_id('uaccg_e').charge = 0
    model.metabolites.get_by_id('uaccg_e').formula = 'C20H29N3O19P2'

    # Change the charge of uamr_c
    model.metabolites.get_by_id('uamr_c').charge = -3

    return model

In [14]:
def fix_tagabc(model):
    
    # Change the formula and charge of tagur_e
    model.metabolites.get_by_id('tagur_e').formula = 'C6H10O7'

    # Change the charge of tagur_c
    model.metabolites.get_by_id('tagur_c').charge = -1

    return model

## Functions to test the models for imbalanced reactions

In [15]:
# Creates a list of all internal reactions in the model

def internal_reactions(model):
    
    # Get all reaction_ids in the model
    model_reactions = []
    for reaction in model.reactions:
        model_reactions.append(reaction.id)

    # Filter out all exchanges from the list of reactions
    indicator_exchanges = ['EX','sink','Growth']
    internal_reactions = list(filter(lambda i: all(indicator not in i for indicator in indicator_exchanges), model_reactions))
    
    return internal_reactions

In [16]:
# Confirms if a single reaction is mass balanced
# This function checks if the sum of the elements in the reactants equals the sum of the elements in the products
# It returns True if the reaction is mass balanced, and False otherwise

def is_mass_balanced(reaction):

    # Create a dict to store the sum of each element in the reaction
    balance = defaultdict(int)

    # Iterate over the metabolites in the reaction 
    for metabolite, coefficient in (model.reactions.get_by_id(reaction).metabolites).items():

        # Check if the metabolite has elements
        if metabolite.elements is None or len(metabolite.elements) == 0:
            return False
        
        # Sum the elements in the balance dict using their coefficients
        for element, amount in (metabolite.elements).items():
            balance[element] += coefficient * amount

    # Returns True if all elements are balanced (in a margin of 1e-10)        
    return all(-1e-10 < amount < 1e-10 for amount in balance.values()) 

In [17]:
# This function checks all internal reactions in the model and returns a list of unbalanced reactions

def find_mass_unbalanced_reactions(model):

    # Get all internal reactions in the model
    reactions = internal_reactions(model)

    # Check each reaction for mass balance using the is_mass_balanced function
    # and return a list of unbalanced reactions
    return [rxn for rxn in reactions if not is_mass_balanced(rxn)]

Dictionary to collect all models that still have imbalanced reactions after the modification 

In [18]:
# Dict to save the models with imbalanced reactions after the modification by the Functions to fix imbalanced reactions
imbalanced_models_aftermod = {}

## Main

If a reaction is imbalanced in a model the corresponding function will be applied to it.  
Afterwards the model gets saved as "modified_{model_id}" and a dictionary containing all modified_models as keys and their still imbalanced reactions (if existing) gets created 

In [24]:
for m in os.listdir(model_path):
    if m.endswith(".xml"):

        print("checking", m)

        # if f'{m[:-4]}_balanced.xml' not in os.listdir(save_path):

        model = read_sbml_model(model_path + m)

        try:
            if m in reaction_dict["SALCHS4FEabcpp"]:
                model = fix_salchs4feabcpp(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["MCBTFabcpp"]:
                model = fix_mcbtfabcpp(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["CMCBTF"]:
                model = fix_cmcbtf(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["DHBSZ3FEabcpp"]:
                model = fix_dhbsz3feabcpp(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["MAN6Gpts"]:
                model = fix_man6gpts(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["UACCpts"]:
                model = fix_uaccpts(model)
        except KeyError:
            pass
        try:
            if m in reaction_dict["TAGabc"]:
                model = fix_tagabc(model)
        except KeyError:
            pass

        # Write the model
        write_sbml_model(model, save_path+f'{m[:-4]}_balanced.xml')

        # Test if the fixes worked
        imbalanced_reactions_aftermod = find_mass_unbalanced_reactions(model)
        # print(imbalanced_reactions_aftermod)
        if len(imbalanced_reactions_aftermod) != 0:
            imbalanced_models_aftermod[m] = imbalanced_reactions_aftermod
        else:
            print("all fixed")

#with open('imbalanced_reactions_roots_modified2.json', 'w') as file:
    #json.dump(imbalanced_reactions_aftermod, file)

checking AA5.xml


Adding exchange reaction EX_14glucan_e with default bounds for boundary metabolite: 14glucan_e.
Adding exchange reaction EX_25dkglcn_e with default bounds for boundary metabolite: 25dkglcn_e.
Adding exchange reaction EX_2m35mdntha_e with default bounds for boundary metabolite: 2m35mdntha_e.
Adding exchange reaction EX_2pglyc_e with default bounds for boundary metabolite: 2pglyc_e.
Adding exchange reaction EX_34dhbz_e with default bounds for boundary metabolite: 34dhbz_e.
Adding exchange reaction EX_35dnta_e with default bounds for boundary metabolite: 35dnta_e.
Adding exchange reaction EX_3mb_e with default bounds for boundary metabolite: 3mb_e.
Adding exchange reaction EX_3oxoadp_e with default bounds for boundary metabolite: 3oxoadp_e.
Adding exchange reaction EX_4abut_e with default bounds for boundary metabolite: 4abut_e.
Adding exchange reaction EX_4hbald_e with default bounds for boundary metabolite: 4hbald_e.
Adding exchange reaction EX_4hbz_e with default bounds for boundary me

all fixed
checking AA4.xml


Adding exchange reaction EX_12ppd__R_e with default bounds for boundary metabolite: 12ppd__R_e.
Adding exchange reaction EX_12ppd__S_e with default bounds for boundary metabolite: 12ppd__S_e.
Adding exchange reaction EX_14glucan_e with default bounds for boundary metabolite: 14glucan_e.
Adding exchange reaction EX_15dap_e with default bounds for boundary metabolite: 15dap_e.
Adding exchange reaction EX_23camp_e with default bounds for boundary metabolite: 23camp_e.
Adding exchange reaction EX_23ccmp_e with default bounds for boundary metabolite: 23ccmp_e.
Adding exchange reaction EX_23cgmp_e with default bounds for boundary metabolite: 23cgmp_e.
Adding exchange reaction EX_23cump_e with default bounds for boundary metabolite: 23cump_e.
Adding exchange reaction EX_25dkglcn_e with default bounds for boundary metabolite: 25dkglcn_e.
Adding exchange reaction EX_26dap__M_e with default bounds for boundary metabolite: 26dap__M_e.
Adding exchange reaction EX_2ameph_e with default bounds for b

all fixed
checking AA6.xml


...

all fixed
checking AA7.xml


Adding exchange reaction EX_12dgr160_e with default bounds for boundary metabolite: 12dgr160_e.
Adding exchange reaction EX_12dgr180_e with default bounds for boundary metabolite: 12dgr180_e.
Adding exchange reaction EX_1ag160_e with default bounds for boundary metabolite: 1ag160_e.
Adding exchange reaction EX_1ag180_e with default bounds for boundary metabolite: 1ag180_e.
Adding exchange reaction EX_1ag181d9_e with default bounds for boundary metabolite: 1ag181d9_e.
Adding exchange reaction EX_1ag182d9d12_e with default bounds for boundary metabolite: 1ag182d9d12_e.
Adding exchange reaction EX_25dkglcn_e with default bounds for boundary metabolite: 25dkglcn_e.
Adding exchange reaction EX_2ameph_e with default bounds for boundary metabolite: 2ameph_e.
Adding exchange reaction EX_2ddglcn_e with default bounds for boundary metabolite: 2ddglcn_e.
Adding exchange reaction EX_2m35mdntha_e with default bounds for boundary metabolite: 2m35mdntha_e.
Adding exchange reaction EX_34dhbz_e with de

all fixed
checking AA2.xml


Ignoring reaction 'EX_12ppd__R_e' since it already exists.
Ignoring reaction 'EX_25dkglcn_e' since it already exists.
Ignoring reaction 'EX_2dhglcn_e' since it already exists.
Ignoring reaction 'EX_2m35mdntha_e' since it already exists.
Ignoring reaction 'EX_2pglyc_e' since it already exists.
Ignoring reaction 'EX_35dnta_e' since it already exists.
Ignoring reaction 'EX_3h4atb_e' since it already exists.
Ignoring reaction 'EX_3mb_e' since it already exists.
Ignoring reaction 'EX_3oxoadp_e' since it already exists.
Ignoring reaction 'EX_4ahmmp_e' since it already exists.
Ignoring reaction 'EX_4hba_e' since it already exists.
Ignoring reaction 'EX_4hbald_e' since it already exists.
Ignoring reaction 'EX_4hbz_e' since it already exists.
Ignoring reaction 'EX_4hoxpac_e' since it already exists.
Ignoring reaction 'EX_4hpro_LT_e' since it already exists.
Ignoring reaction 'EX_4oxptn_e' since it already exists.
Ignoring reaction 'EX_5drib_e' since it already exists.
Ignoring reaction 'EX_5mdr

all fixed


To test if there are still models with imbalanced reactions. If this outputs only {}, then all models are mass balanced.

In [25]:
print(imbalanced_models_aftermod)

{}


## Memote Reports

In [29]:
def memote_report(model_x, model_path_x, report_filename_x):
    print(f"Running memote for {model_x}")
    subprocess.run(["memote", "report", "snapshot", "--filename", report_filename_x, model_path_x, "--solver", "cplex"])

In [30]:
# Run memote for each model (~1-2min PER report)
for model in (f for f in os.listdir(save_path) if f.endswith(".xml")):
    # path to one specific model
    model_path = f"{save_path}/{model}"
    # path where reports should be stored and report file name
    if "AA3" in model:
        report_filename = f"Reports/mass_balance/report_{model[:3]}_balanced.html"
    else:
        report_filename = f"Reports/mass_balance/report_{model[:3]}_balanced.html"

    memote_report(model, model_path, report_filename)
print("done")

Running memote for AA1_balanced.xml
Running memote for AA1_balanced.xml
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.FFFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py . [ 67%]
..FFFFF......s..FFF.FF.F.FFF                              

Writing snapshot report to 'Reports/mass_balance/report_AA1_balanced.html'.


Running memote for AA3_balanced.xml
Running memote for AA3_balanced.xml
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.FFFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
F.......F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py . [ 67%]
.............s...FFFFF.F.FFF                              

Writing snapshot report to 'Reports/mass_balance/report_AA3_balanced.html'.


Running memote for AA5_balanced.xml
Running memote for AA5_balanced.xml
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.FFFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py . [ 67%]
..FFFFFFFFFFFFFFFFF.FF.F.FFF                              

...

Running memote for AA4_balanced.xml
Running memote for AA4_balanced.xml
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F...........FFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py . [ 67%]
.............ss..FF.FF.F.FFF                              

Writing snapshot report to 'Reports/mass_balance/report_AA4_balanced.html'.


Running memote for AA6_balanced.xml
Running memote for AA6_balanced.xml
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F..FFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py . [ 67%]
..FFFFF......s..FFF.FF.F.FFF                              

Writing snapshot report to 'Reports/mass_balance/report_AA6_balanced.html'.


done
